## Tutorial on face detection with webcam

Relevant websites:
https://iotdesignpro.com/projects/face-recognition-door-lock-system-using-raspberry-pi
https://github.com/mowshon/age-and-gender
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/object_detection_camera.html
https://learnopencv.com/training-a-custom-object-detector-with-dlib-making-gesture-controlled-applications/
http://dlib.net/ml.html

Haar cascades, face and eyes
https://towardsdatascience.com/computer-vision-detecting-objects-using-haar-cascade-classifier-4585472829a9

Raspberry pi and others:
https://qengineering.eu/deep-learning-with-raspberry-pi-and-alternatives.html

### Raspberry PI camera
you need to run the following code so that opencv can get the image with `VideoCapture`:

`sudo modprobe bcm2835-v4l2`

See:
https://stackoverflow.com/questions/29583533/videocapture-open0-wont-recognize-pi-cam/37530016#37530016

Make also sure the interface for (legacy) video is enabled in the raspberry Pi configuration `sudo raspi-config` -> 'Interfacing Options' -> 'Camera'


In [ ]:
import cv2
#import sys
import os
import json
import numpy as np

# pip install numpy
# pip install opencv-python
# pip install opencv-contrib-python

In [ ]:
def test_webcam(mirror=False, width=600, height=600):
    """ Basic webcam display function, to test the webcam
    """
    cam = cv2.VideoCapture(0)
    # Check if the webcam is opened correctly
    if not cam.isOpened():
        raise IOError("Cannot open webcam")
    print("[INFO] Press esc to quit")
    while True:
        ret_val, img = cam.read()
        if mirror: 
            img = cv2.flip(img, 1)
        ## display image
        try:
            cv2.imshow('my webcam', img)
            cv2.namedWindow('my webcam',cv2.WINDOW_NORMAL)
            cv2.resizeWindow('my webcam', width, height)
            if cv2.waitKey(1) == 27: 
                break  # esc to quit
        except:
            cam.release()
            cv2.destroyAllWindows()
            raise ValueError('Problem displaying the video.')
    cam.release()
    cv2.destroyAllWindows()

In [ ]:
test_webcam()

Training face recognition taken from: (initially for Raspberry pi)
[https://iotdesignpro.com/projects/face-recognition-door-lock-system-using-raspberry-pi](https://iotdesignpro.com/projects/face-recognition-door-lock-system-using-raspberry-pi)

There are 3 steps:

* Image aquisition, the webcam takes pictures of the face and save them in a data folder
* Training on the faces in the dataset
* face recognition


## Initialization

In [ ]:
def create_dir(dirName):
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")

In [ ]:
# detect the current working directory and print it
path = os.getcwd()
data_dir = os.path.join(path,"dataset")
trained_model_dir = os.path.join(path,"trained_models")
# Create target Directory if don't exist
create_dir(data_dir)
create_dir(trained_model_dir)
# initializing face recognition methods
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
model_file = os.path.join(trained_model_dir,'trainer.yml')
person_names_file = os.path.join(trained_model_dir,"persons.json")
confidence_threshold = 10

In [ ]:
def get_faces(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #gray = cv2.equalizeHist(gray)
    W, H = gray.shape
    minW, minH = int(0.1* W), int(0.1* H) # 10% of image size
    faces = face_detector.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (minW, minH),
       )
    return faces,gray

## Image acquisition

In [ ]:
# import cv2

cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
# For each person, enter a name
face_id = input('\n enter user name end press <return> ==>  ')
print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0
while(True):
    ret, img = cam.read()
    img = cv2.flip(img, 1) # mirror
    faces, gray = get_faces(img)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        data_file = "dataset/User." + str(face_id) + '.' + str(count) + ".jpg"
        print("saving file", data_file)
        cv2.imwrite(data_file, gray[y:y+h,x:x+w])
    cv2.imshow('image', img)
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 30: # Take 30 face sample and stop video
         break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

## Training

In [ ]:
#import numpy as np
# import os
# import cv2

# function to get the images and label data
def getImagesAndLabels(data_dir):
    imagePaths = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]     
    faceSamples=[]
    ids = []
    person_name_dic = {}
    id_count = 0
    for imagePath in imagePaths:
        img = cv2.imread(imagePath, 0) # open in grayscale
        img_numpy = np.array(img,'uint8')
        person_name = os.path.split(imagePath)[-1].split(".")[1]
        if person_name in person_name_dic:
            person_id = person_name_dic[person_name]
        else:
            person_id = id_count
            person_name_dic[person_name]= person_id
            id_count +=1
        faces = face_detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(person_id)
    return faceSamples,ids, person_name_dic


In [ ]:
#import json

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces, ids, person_name_dic = getImagesAndLabels(data_dir)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write(model_file) # recognizer.save() worked on Mac, on Ubuntu, but not on Pi?
# Print the number of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))
# save the name of the persons
person_id2name = {value : key for (key, value) in person_name_dic.items()}
with open(person_names_file, 'w') as fp:
    json.dump(person_id2name, fp,  indent=4)

## Detection

In [ ]:
#import cv2
#import json

recognizer.read(model_file)
with open(person_names_file, 'r') as fp:
    person_id2name = json.load(fp)
font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

while True:
    ret, img =cam.read()
    img = cv2.flip(img, 1) # mirror
    faces,gray = get_faces(img)
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        person_id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        # Check if confidence is less them 100 ==> "0" is perfect match 
        if ((100 - confidence) > confidence_threshold):
            person_name = person_id2name[str(person_id)]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            person_name = "Unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        cv2.putText(img, str(person_name), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    cv2.imshow('camera',img)
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

In [ ]:
cam.release()
cv2.destroyAllWindows()